## Problem 4

In [51]:
1.6381e+08

163810000.0

### 讀取資料

In [22]:
from gurobipy import *
import pandas as pd
import numpy as np

In [23]:
# Read excel sheets and transform them into lists and matrices
demand_info=pd.read_excel('OR109-1_case01_data.xlsx','Demand',index_col = 0)
initial_inventory_info=pd.read_excel('OR109-1_case01_data.xlsx','Initial inventory',index_col = 0)
shipping_cost_info=pd.read_excel('OR109-1_case01_data.xlsx','Shipping cost',index_col = 0)
intransit=pd.read_excel('OR109-1_case01_data.xlsx','In-transit',index_col = 0)
size_info=pd.read_excel('OR109-1_case01_data.xlsx','Size',index_col = 0)
price_and_cost_info=pd.read_excel('OR109-1_case01_data.xlsx','Price and cost',index_col = 0)
shortage_info=pd.read_excel('OR109-1_case01_data.xlsx','Shortage',index_col = 0)

In [24]:
size=size_info['Cubic meter'].values.tolist()

In [25]:
shipping_cost_info['Ocean_freight']=0

In [26]:
shipping_cost=shipping_cost_info.values.tolist()

In [27]:
backorder=shortage_info['Backorder'].values.tolist()
lost_sales=shortage_info['Lost sales'].values.tolist()
backorder_percentage=shortage_info['Backorder percentage'].values.tolist()

In [28]:
backorder

[535, 250, 1345, 980, 345, 1575, 705, 820, 1650, 930]

In [29]:
lost_sales

[5700, 3000, 17900, 10600, 4900, 22500, 7100, 11400, 24000, 11600]

In [30]:
backorder_percentage

[0.0, 0.7, 0.1, 1.0, 1.0, 0.3, 0.6, 0.2, 0.1, 0.5]

In [31]:
demand=demand_info.values.tolist()
initial_inventory=initial_inventory_info['Initial inventory'].values.tolist()
express_delivery_cost=shipping_cost_info['Express delivery'].values.tolist()
air_freight_cost=shipping_cost_info['Air freight'].values.tolist()
ocean_freight_cost=[0]*10

In [32]:
purchasing_cost=price_and_cost_info['Purchasing cost'].values.tolist()
holding_cost=price_and_cost_info['Holding'].values.tolist()
fix_cost=[100,80,50]

In [33]:
intransit.insert(loc=2, column='6', value=0)
intransit.insert(loc=3, column='7', value=0)
intransit.insert(loc=4, column='8', value=0)
intransit.insert(loc=0, column='3', value=initial_inventory)
intransit=intransit.values.tolist()

In [34]:
intransit

[[800, 0, 0, 0, 0, 0],
 [600, 48, 0, 0, 0, 0],
 [425, 0, 20, 0, 0, 0],
 [350, 153, 0, 0, 0, 0],
 [400, 0, 0, 0, 0, 0],
 [524, 18, 23, 0, 0, 0],
 [453, 28, 45, 0, 0, 0],
 [218, 0, 0, 0, 0, 0],
 [673, 109, 34, 0, 0, 0],
 [200, 0, 0, 0, 0, 0]]

In [35]:
product_len=range(10)
month_len=range(6)
shipping_method_len=range(3)

In [36]:
# Big number M
M=np.sum(demand)

### 模型

In [45]:
cs1 = Model("cs1")

In [46]:
#定義variable
#定義x_ijt
x = []
for i in product_len:
    x.append([])
    for j in shipping_method_len:
        x[i].append([])
        for t in month_len:
            x[i][j].append(cs1.addVar(lb = 0, vtype = GRB.CONTINUOUS, 
                                      name = "x" +  str(i+1) +'_'+ str(j+1)+'_'+ str(t+1))) 
            
#定義v_it
v = []
for i in product_len:
    v.append([])
    for t in month_len:
        v[i].append(cs1.addVar(lb = 0, vtype = GRB.CONTINUOUS, 
                                  name = "v" +  str(i+1)+ '_'+ str(t+1))) 
        
#定義y_jt
y = []
for j in shipping_method_len:
    y.append([])
    for t in month_len:
        y[j].append(cs1.addVar(lb = 0, vtype = GRB.BINARY, 
                                  name = "y" +  str(j+1)+ '_'+ str(t+1))) 
        
#定義z
z = []
for t in month_len:
    z.append(cs1.addVar(lb = 0, vtype = GRB.INTEGER, 
                              name = "z" +'_'+ str(t+1))) 
    
#定義s_it
s = []
for i in product_len:
    s.append([])
    for t in month_len:
        s[i].append(cs1.addVar(lb = 0, vtype = GRB.CONTINUOUS,
                                  name = "s" +  str(i+1)+ '_'+ str(t+1))) 
        
#定義k_it
k = []
for i in product_len:
    k.append([])
    for t in month_len:
        k[i].append(cs1.addVar(lb = 0, vtype = GRB.BINARY, 
                                  name = "k" +  str(i+1)+ '_'+ str(t+1))) 
        
#定義D_it
D = []
for i in product_len:
    D.append([])
    for t in month_len:
        D[i].append(cs1.addVar(lb = 0, vtype = GRB.CONTINUOUS,
                                  name = "D" +  str(i+1)+ '_'+ str(t+1))) 

In [47]:
# setting the objective function 
cs1.setObjective(
    
    np.sum( [[v[i][t]*holding_cost[i] for i in product_len] for t in month_len])+  # inventory*holding cost
    np.sum([(np.array(shipping_cost)+np.array(purchasing_cost)[:,None])*np.array(x)[:,:,t] for t in month_len])+ #x's shipping and purchasing cost
    np.sum(np.array(y)*np.array(fix_cost)[:,None])+ # method's fix cost
    np.sum(np.array(z)*2750 )+ # container  cost
    np.sum(((1-np.array(backorder_percentage))[:,None]*np.array(lost_sales)[:,None])*s)+ #cost of lost sales
    np.sum((np.array(backorder_percentage)[:,None]*np.array(backorder)[:,None])*s) #cost of back order
    
    , GRB.MINIMIZE) 

In [48]:
cs1.addConstrs( ( v[i][t]-s[i][t] == v[i][t-1]+ (np.array(x)[i,:,t]).sum()+ intransit[i][t]-D[i][t]
                 for i in product_len for t in month_len if t>0), "inventory demand equation" )
cs1.addConstrs( ( v[i][t]-s[i][t] == (np.array(x)[i,:,t]).sum()+ intransit[i][0]-D[i][t]
                 for i in product_len for t in month_len if t==0), "inventory demand equation first round" )

cs1.addConstrs( ( M*k[i][t] >= s[i][t] for i in product_len for t in month_len), "k is bigger than s" )
cs1.addConstrs( ( M*(1-k[i][t]) >= v[i][t] for i in product_len for t in month_len), "1-k is bigger than v" )
cs1.addConstrs( ( M*y[j][t] >= (np.array(x)[:,j,t]).sum()
                 for j in shipping_method_len for t in month_len), "y is bigger than x" )
cs1.addConstrs( ( z[t] >= ((( np.array(x)[:,2,:] * np.array(size)[:,None] ).sum(axis=0) )/30) [t] for t in month_len)
               , "z calculate container number" )

cs1.addConstrs( ( D[i][t] >= demand[i][t]+ backorder_percentage[i]*s[i][t-1] 
                 for i in product_len for t in month_len if t>0), "D >= D" )
cs1.addConstrs( ( D[i][t] >= demand[i][t]+ backorder_percentage[i]*0
                 for i in product_len for t in month_len if t==0), "D >= D first round" )

{(0, 0): <gurobi.Constr *Awaiting Model Update*>,
 (1, 0): <gurobi.Constr *Awaiting Model Update*>,
 (2, 0): <gurobi.Constr *Awaiting Model Update*>,
 (3, 0): <gurobi.Constr *Awaiting Model Update*>,
 (4, 0): <gurobi.Constr *Awaiting Model Update*>,
 (5, 0): <gurobi.Constr *Awaiting Model Update*>,
 (6, 0): <gurobi.Constr *Awaiting Model Update*>,
 (7, 0): <gurobi.Constr *Awaiting Model Update*>,
 (8, 0): <gurobi.Constr *Awaiting Model Update*>,
 (9, 0): <gurobi.Constr *Awaiting Model Update*>}

In [49]:
cs1.optimize()

Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (mac64)
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 264 rows, 444 columns and 1019 nonzeros
Model fingerprint: 0x0cb8ee86
Variable types: 360 continuous, 84 integer (78 binary)
Coefficient statistics:
  Matrix range     [2e-04, 7e+03]
  Objective range  [4e+01, 2e+04]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+01, 7e+03]
Found heuristic solution: objective 3.961236e+07
Presolve removed 80 rows and 89 columns
Presolve time: 0.00s
Presolved: 184 rows, 355 columns, 838 nonzeros
Found heuristic solution: objective 3.865694e+07
Variable types: 281 continuous, 74 integer (68 binary)

Root relaxation: objective 1.357289e+07, 83 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 1.3573e+07    0   10 3.8657e+07 1.3573e+07  64.9%     -    0s
H   

In [50]:
print("z* =", cs1.objVal)    # print objective value

z* = 13576243.57142857


In [43]:
x

[[[<gurobi.Var x1_1_1 (value 0.0)>,
   <gurobi.Var x1_1_2 (value 0.0)>,
   <gurobi.Var x1_1_3 (value 0.0)>,
   <gurobi.Var x1_1_4 (value 0.0)>,
   <gurobi.Var x1_1_5 (value 0.0)>,
   <gurobi.Var x1_1_6 (value 0.0)>],
  [<gurobi.Var x1_2_1 (value 0.0)>,
   <gurobi.Var x1_2_2 (value 0.0)>,
   <gurobi.Var x1_2_3 (value 0.0)>,
   <gurobi.Var x1_2_4 (value 0.0)>,
   <gurobi.Var x1_2_5 (value 0.0)>,
   <gurobi.Var x1_2_6 (value 38.0)>],
  [<gurobi.Var x1_3_1 (value 0.0)>,
   <gurobi.Var x1_3_2 (value 0.0)>,
   <gurobi.Var x1_3_3 (value 0.0)>,
   <gurobi.Var x1_3_4 (value 0.0)>,
   <gurobi.Var x1_3_5 (value 0.0)>,
   <gurobi.Var x1_3_6 (value 0.0)>]],
 [[<gurobi.Var x2_1_1 (value 0.0)>,
   <gurobi.Var x2_1_2 (value 0.0)>,
   <gurobi.Var x2_1_3 (value 0.0)>,
   <gurobi.Var x2_1_4 (value 0.0)>,
   <gurobi.Var x2_1_5 (value 0.0)>,
   <gurobi.Var x2_1_6 (value 0.0)>],
  [<gurobi.Var x2_2_1 (value 0.0)>,
   <gurobi.Var x2_2_2 (value 0.0)>,
   <gurobi.Var x2_2_3 (value 0.0)>,
   <gurobi.Var x2_2_4 